<a href="https://colab.research.google.com/github/iamvinitk/NER-CMPE-257/blob/master/code_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

with open('Programming_languages.json', 'r') as file:
    programming_languages = json.load(file)['PROGRAMMING_LANGUAGES']

In [ ]:
# programming_languages
from bs4 import BeautifulSoup
import re

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text
# cleaned_text = clean_text(text_data)

In [ ]:
data_folder = '..\data'
text_data = []
for filename in os.listdir(data_folder):
    with open(os.path.join(data_folder, filename), 'r') as file:
        text_data.append(clean_text(file))

In [ ]:
len(text_data)

10230

In [ ]:
# pip install nltk scikit-learn matplotlib wordcloud

In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
# !python -m pip install --user --upgrade pip

In [ ]:
# !pip cache purge
# !pip install spacy

In [ ]:
# !python -m spacy download en_core_web_sm

In [ ]:
# import spacy
# import os

# # Get the directory containing the about.py file
# about_dir = os.path.dirname(spacy.about.__file__)

# # Specify the path to the en_core_web_sm model directory
# model_path = os.path.join(about_dir, 'data', 'en_core_web_sm')

# print(model_path)

In [ ]:
# import spacy

# # Load pre-trained spaCy model
# nlp = spacy.load('en_core_web_sm')

# def extract_entities(text):
#     doc = nlp(text)
#     entities = [(ent.text, ent.label_) for ent in doc.ents]
#     return entities

# # Example usage
# entities = extract_entities(text_data)

In [ ]:
!pip install stanfordnlp>=0.1.0,<0.2.0

The system cannot find the file specified.


In [ ]:
import stanfordnlp
def process_with_corenlp(text):
    # Download the pre-trained model (if not already downloaded)
    # stanfordnlp.download('en')  # Downloads English models
    # Initialize the pipeline
    nlp = stanfordnlp.Pipeline()
    # Process the text
    doc = nlp(text)
    # Extract named entities
    entities = []
    for sentence in doc.sentences:
        for ent in sentence.ents:
            entities.append((ent.text, ent.type))
    return entities

In [ ]:
# !pip install stanfordnlp --upgrade

In [ ]:
# !pip install torch --upgrade

In [ ]:
entities = process_with_corenlp(text_data[0])
entities

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords(text_data):
    # Concatenate the strings in text_data into a single string
    concatenated_text = ' '.join(text_data)
    # Use TfidfVectorizer with custom stop words
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([concatenated_text])
    # Get feature names (words)
    feature_names = vectorizer.get_feature_names_out()
    # Identify keywords using TF-IDF scores
    tfidf_sum_per_word = tfidf_matrix.sum(axis=0).A1
    keywords = [feature_names[i] for i in tfidf_sum_per_word.argsort()[::-1][:10]]  # Extract top 10 keywords
    return keywords

# Example usage
keywords = extract_keywords(text_data)

In [ ]:
keywords

['experience',
 'work',
 'team',
 'data',
 'business',
 'including',
 'skills',
 'role',
 'teams',
 'status']

In [ ]:
def visualize_keywords(keywords):
    wordcloud = WordCloud(width=800, height=400, max_font_size=110).generate(' '.join(keywords))
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

# Example usage


In [ ]:
# visualize_keywords(keywords)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(resume, job_description):
    documents = [resume, job_description]

    # Convert the documents into a matrix of TF-IDF features
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Compute the cosine similarity between the two documents
    similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

    return similarity_score

# Example usage:
resume_text = "Experienced software engineer with strong programming skills..."
job_description_text = "We are seeking a highly skilled software engineer..."

score = calculate_similarity(resume_text, job_description_text)
print(f"Similarity Score: {score}")


Similarity Score: 0.16839750037215276


In [8]:
import spacy

def calculate_similarity(resume, job_description):
    nlp = spacy.load("en_core_web_sm")

    # Process the text using spaCy
    resume_doc = nlp(resume)
    job_description_doc = nlp(job_description)

    # Calculate the similarity between the processed texts
    similarity_score = resume_doc.similarity(job_description_doc)

    return similarity_score

# Example usage:
resume_text = "Experienced software engineer with strong programming skills..."
job_description_text = "We are seeking a highly skilled software engineer..."

score = calculate_similarity(resume_text, job_description_text)
print(f"Similarity Score: {score}")

Similarity Score: 0.5396360952066532


<ipython-input-8-e888ebcc57b2>:11: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = resume_doc.similarity(job_description_doc)


Similarity score increases a lot after using spacy

In [10]:
pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 818.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3183168109be9ba20a1ed43aa9db0d164bcb359f1ec1c05b09922c661227a462
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [11]:
from sentence_transformers import SentenceTransformer, util

def calculate_similarity(resume, job_description):
    model_name = "paraphrase-MiniLM-L6-v2"
    model = SentenceTransformer(model_name)

    # Encode the resume and job description
    resume_embedding = model.encode(resume, convert_to_tensor=True)
    job_description_embedding = model.encode(job_description, convert_to_tensor=True)

    # Calculate cosine similarity between the embeddings
    similarity_score = util.pytorch_cos_sim(resume_embedding, job_description_embedding)[0][0].item()

    return similarity_score

# Example usage:
resume_text = "Experienced software engineer with strong programming skills..."
job_description_text = "We are seeking a highly skilled software engineer..."

score = calculate_similarity(resume_text, job_description_text)
print(f"Similarity Score: {score}")


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Similarity Score: 0.7389307618141174


In [12]:
from sentence_transformers import SentenceTransformer, util

def read_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def calculate_similarity(resume_path, job_description_path, threshold=0.8):
    model_name = "paraphrase-MiniLM-L6-v2"
    model = SentenceTransformer(model_name)

    # Read the documents from file
    resume = read_document(resume_path)
    job_description = read_document(job_description_path)

    # Encode the resume and job description
    resume_embedding = model.encode(resume, convert_to_tensor=True)
    job_description_embedding = model.encode(job_description, convert_to_tensor=True)

    # Calculate cosine similarity between the embeddings
    similarity_score = util.pytorch_cos_sim(resume_embedding, job_description_embedding)[0][0].item()

    # Classify the match based on the threshold
    match_classification = "Good Match" if similarity_score >= threshold else "Not a Good Match"

    # Extract and display the most relevant sentences
    resume_sentences = [sent.text for sent in model.tokenize(resume) if similarity_score >= threshold]
    job_description_sentences = [sent.text for sent in model.tokenize(job_description) if similarity_score >= threshold]

    return {
        "Similarity Score": similarity_score,
        "Match Classification": match_classification,
        "Relevant Resume Sentences": resume_sentences,
        "Relevant Job Description Sentences": job_description_sentences
    }

# Example usage:
resume_path = "resume.txt"
job_description_path = "job_description.txt"

result = calculate_similarity(resume_path, job_description_path)
for key, value in result.items():
    print(f"{key}: {value}")

Similarity Score: 0.733249306678772
Match Classification: Not a Good Match
Relevant Resume Sentences: []
Relevant Job Description Sentences: []


Updated code with original documents(one job description and a resume)

**paraphrase-MiniLM-L6-v2** model helps a lot by increasing similarity score

In [13]:
pip install tk


In [15]:
import tkinter as tk
from tkinter import filedialog
from sentence_transformers import SentenceTransformer, util

class DocumentSimilarityGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Document Similarity Checker")

        self.model_name = "paraphrase-MiniLM-L6-v2"
        self.model = SentenceTransformer(self.model_name)

        self.create_widgets()
def create_widgets(self):
        tk.Label(self.root, text="Select Resume:").pack()
        self.resume_button = tk.Button(self.root, text="Browse", command=self.browse_resume)
        self.resume_button.pack()

        tk.Label(self.root, text="Select Job Description:").pack()
        self.job_description_button = tk.Button(self.root, text="Browse", command=self.browse_job_description)
        self.job_description_button.pack()

        self.match_button = tk.Button(self.root, text="Check Similarity", command=self.calculate_similarity)
        self.match_button.pack()

        self.result_label = tk.Label(self.root, text="")
        self.result_label.pack()

    def browse_resume(self):
        resume_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
        if resume_path:
            self.resume_button.config(text=f"Resume: {resume_path}")

        self.resume_path = resume_path

    def browse_job_description(self):
        job_description_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
        if job_description_path:
            self.job_description_button.config(text=f"Job Description: {job_description_path}")

        self.job_description_path = job_description_path

    def calculate_similarity(self):
        if hasattr(self, 'resume_path') and hasattr(self, 'job_description_path'):
            # Read the documents from file
            resume = self.read_document(self.resume_path)
            job_description = self.read_document(self.job_description_path)

            # Encode the resume and job description
            resume_embedding = self.model.encode(resume, convert_to_tensor=True)
            job_description_embedding = self.model.encode(job_description, convert_to_tensor=True)

            # Calculate cosine similarity between the embeddings
            similarity_score = util.pytorch_cos_sim(resume_embedding, job_description_embedding)[0][0].item()

            # Display the result
            match_classification = "Good Match" if similarity_score >= 0.8 else "Not a Good Match"
            result_text = f"Similarity Score: {similarity_score:.2f}\nMatch Classification: {match_classification}"
            self.result_label.config(text=result_text)
        else:
            self.result_label.config(text="Please select both a resume and a job description.")

    def read_document(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()


if __name__ == "__main__":
    root = tk.Tk()
    app = DocumentSimilarityGUI(root)
    root.mainloop()


In [6]:
import json
import os
import re
from langdetect import detect

In [ ]:
DATA_DIR = '../job-descriptions/'
OUTPUT_DIR = '../software-jobs/'

In [1]:
files = os.listdir(DATA_DIR)
print("Number of files: ", len(files))

In [3]:
job_descriptions = []
for file in files:
    if ".json" in file:
        with open(DATA_DIR + file, 'r') as f:
            data = json.load(f)
            data['content'] = re.sub(r'\n+', '\n', data['content'])  # remove multiple newlines
            data['content'] = re.sub(r'\t+', '\t', data['content'])  # remove multiple tabs
            data['content'] = re.sub(r'\r+', '\r', data['content'])  # remove multiple carriage returns
            data['content'] = re.sub(r'\r\n+', '\r\n', data['content'])  # remove multiple carriage returns
            data['content'] = re.sub(r'\n\r+', '\n\r', data['content'])  # remove multiple carriage returns
            data['file'] = file
            try:
                if detect(data['content']) == 'en':
                    job_descriptions.append(data)
            except:
                print("Error in file: ", file)
                pass

In [4]:
job_titles = [job['title'] for job in job_descriptions]

In [5]:
# split job titles into words
job_titles_words = []
for title in job_titles:
    job_titles_words.extend(title.split(' '))